# Домашнее задание №2

### Получаем векторные представления новостей

In [1]:
import pandas as pd

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [5]:
# from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [8]:
# предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#! pip install razdel

import pymorphy2  # pip install pymorphy2

In [9]:
import nltk
nltk.download('stopwords')

stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [11]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [12]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

c:\Temp/ipykernel_12888/3066725202.py:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 38.4 s


In [13]:
%%time
#Запускаем лемматизацию текста. 
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 6min 1s


In [15]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [16]:
common_dictionary[7]

'близкий'

In [17]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min


In [18]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [19]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.7587486), (3, 0.035506558), (8, 0.14599323), (17, 0.04108475)]

In [20]:
x=lda.show_topics(num_topics=25, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: это который всё свой мочь год весь
topic_1: египет предприниматель миллиард прогнозировать лекарство индия опасаться
topic_2: российский россия это страна который год система
topic_3: украина погибнуть киев гражданин украинский народный фронт
topic_4: военный газ год армия двигатель который компания
topic_5: сша рак агентство фестиваль операция берег удар
topic_6: год это который новый проект работа россия
topic_7: белоруссия тело белорусский бизнесмен виза железный нож
topic_8: достигать захватить изучение величина франция палатка сибирский
topic_9: восток следователь изучение убийство страдать юг сон
topic_10: предполагаться фильм законопроект парламент мэй поправка подразделение
topic_11: статья километр иск оборудование суд бомба девочка
topic_12: nn год банк который компания это также
topic_13: исследование научный журнал риск это проверка связанный
topic_14: год который смерть совет наука свой комиссия
topic_15: путин эксперимент сотрудник задержать полиция чиновник земл

In [21]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [22]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.051670,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,4896,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.062106,...,0.0,0.413417,0.000000,0.0,0.0,0.000000,0.435060,0.000000,0.0,0.0
2,4897,0.758690,0.000000,0.0,0.035406,0.0,0.0,0.000000,0.0,0.146003,...,0.0,0.000000,0.041235,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,4898,0.580426,0.034191,0.0,0.000000,0.0,0.0,0.071635,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.022767,0.125529,0.119551,0.0,0.0
4,4899,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.070647,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [23]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [24]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [25]:
doc_dict[293622]

array([0.2863383 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.02339354, 0.        , 0.        , 0.03968896,
       0.        , 0.        , 0.        , 0.02868813, 0.20771892,
       0.06368998, 0.        , 0.        , 0.01861974, 0.        ,
       0.0200288 , 0.        , 0.07915061, 0.22354834, 0.        ])

## Задание

Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана.  
Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их:  
roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

Добавим к лекционной функции get_user_embedding еще один аргумент, в котором будем передавать обобщающий метод (mean, median или max)

In [26]:
def get_user_embedding(user_articles_list, method):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = method(user_vector, 0)
    return user_vector

In [27]:
def get_user_embeddings(data, method):
    user_embeddings = pd.DataFrame([i for i in data.apply(lambda x: get_user_embedding(x, method), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    return user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [28]:
target = pd.read_csv("../materials/lection/users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

Получим разбиения train test для различных методов получения embeddings (mean, median, max)

In [30]:
def get_trains_tests(users, target, method_list=[np.mean, np.median, np.max]):
    trains_tests = []
    for method in method_list:
        user_embeddings = get_user_embeddings(users['articles'], method)
        X = pd.merge(user_embeddings, target, 'left')
        X_train, X_test, y_train, y_test = train_test_split(
            X[['topic_{}'.format(i) for i in range(25)]], 
            X['churn'], random_state=0
        )

        trains_tests += [(X_train, X_test, y_train, y_test, method.__name__,)]
    return trains_tests


In [31]:
train_tests = get_trains_tests(users, target)

In [32]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

results = []

for train_test in train_tests:
    X_train, X_test, y_train, y_test, method_name = train_test

    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)

    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)

    roc_auc = roc_auc_score(y_true=y_test, y_score=preds)

    results += [roc_auc, precision[ix], recall[ix], fscore[ix], thresholds[ix]]

    print('Method %s: ROC_AUC=%f Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (method_name,
                                                                            roc_auc,
                                                                            thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))


Method mean: ROC_AUC=0.932940 Best Threshold=0.258762, F-Score=0.638, Precision=0.588, Recall=0.698
Method median: ROC_AUC=0.947136 Best Threshold=0.268904, F-Score=0.667, Precision=0.592, Recall=0.763
Method amax: ROC_AUC=0.976366 Best Threshold=0.357009, F-Score=0.813, Precision=0.839, Recall=0.788


In [33]:
results_df = pd.DataFrame(results)

In [34]:
results_df.to_csv('./comparation_results.csv')

# Выводы

В случае с медианой мы видим достаточно высокое значение метрик при самом низко пороге.  
Если в задаче требуется увеличить Recall, то медиану однозначно стоит рассмотреть в первую очередь для рассчета embedding.  
Если в задаче требуется увеличить Precision, то наибольшее значение при расчете embedding показывает функция max.  
